<a href="https://colab.research.google.com/github/Mario-uni/DAIProLi/blob/main/DAIProLi_6_esp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Determinación de Afinidad de Interacción entre Proteína y Ligando



In [ ]:
#@title Librerias y funciones

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import csv
import zipfile

from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, output_file, save
from bokeh.models import Legend
from bokeh.palettes import linear_palette, Viridis256
from bokeh.palettes import Category20

from mpl_toolkits.mplot3d import Axes3D

from scipy.optimize import curve_fit, leastsq, least_squares
from google.colab import files
from datetime import datetime
import os

# Esto es temporal, la solución final sería subir el modulo al github de Unizar
!git clone https://github.com/Mario-uni/DAIProLi.git
from DAIProLi.funcionesGenerales import  procesa, argLeastSquares

##Funciones específicas

def leeFichero(nombrFich, colValLabel='Wavelength nm.', intercaladas=True, separador=','):
  # Abrimos el fichero
  with open(nombrFich,'r') as fichero:
    # Inicializamos las variables
    col_names = []
    data_rows = []
    #row_names = []

    # Iteramos por las lineas del fichero
    for linea in fichero:
      row = []

      # Si esta línea es la linea que contiene la colValLabel, extraemos los nombres/valores de las columna
      if linea.split(separador)[0] == colValLabel:
        col_names = [dato.strip() for dato in linea.split(separador) if dato.strip()]
      else:
          # Procesamos cada fila de datos
          values=[dato.strip() for dato in linea.split(separador) if dato.strip()]

          if values:
            #row_name = values[0] # El primer valor es el nombre de la fila (Wavelength)
            row_values = values # El resto de los valores lo ponemos como datos

            # Si intercaladas es cierto, tomamos los valores en posiiones pares de row_values
            if intercaladas:
              row_values=row_values[::2]

            # Intentamos convertir cada valor en float, pero nos quedamos con string si no es possible
            processed_row=[]
            for dato in row_values:
              try:
                processed_row.append(float(dato)) # Intentamos convertir en float
              except ValueError:
                processed_row.append(dato) # Nos quedamos el string si no es in float

            #row_names.append(row_name) # Recogemos los nombres de las filas
            data_rows.append(processed_row) # Recogemos los datos de las filas

  # Creamos un DataFrame para los datos recogidos
  df = pd.DataFrame(data_rows)

  # Ponemos los col_names como nombres de las columna, asegurandonos que su numero coincide con el
  # número de columnas en los datos.
  if len(col_names)== len(df.columns) :
    df.columns = col_names # Nos saltamos la primera etiqueta (que es para los nombre de las filas) REVISAR
  else:
    print(f"Warning: The number of columns in the data does not match the header size")
  return df


def preprocesaAbsorbance(df):
  #df=datos
  # Extraemos los valores de la primera columna (Wavelength nm.)
  wavelength= df.iloc[:,0] # La primer columna contiene los valores de longitud de onda

  # Extramemos las medidas de absorbancia (todas las columnas menos la primera)
  absorbance= df.iloc[:,1:]

  # Extraemos los nombres de las columnas a partir de la tercera (incluida)
  volume= df.columns[2:]
  volume = pd.to_numeric(volume)

  # Inicializamos las listas para almacenar los mínimos, máximos y sus respectivas longitudes de onda
  min_vals = []
  max_vals = []
  min_wave = []
  max_wave = []

  # Iteramos sobre cada columna (excepto la primera y la segunda, longitud de onda y baseline respectivamente)
  for col in df.columns[2:]:
    # Encontramos el valor mínimo, máximo y su correspondiente longitud de onda
      min_val = df[col].min()
      max_val = df[col].max()

      min_wl = df[df[col] == min_val]['Wavelength nm.'].iloc[0]
      max_wl = df[df[col] == max_val]['Wavelength nm.'].iloc[0]

      # Almacenamos en las listas
      min_vals.append(min_val)
      max_vals.append(max_val)
      min_wave.append(min_wl)
      max_wave.append(max_wl)


  #Calculamos el primedio de las longitudes de onda correpondientes a los mínimos y máximos
  avg_min_wave = np.mean(min_wave)
  avg_max_wave = np.mean(max_wave)

  # Redondeamos las longitudes al incremente más cercano de 0.5 nm
  avg_min_wave = round(avg_min_wave *2)/2 # Asegurando incrementos de 0.5 nm
  avg_max_wave = round(avg_max_wave *2)/2



  # Encontramos las filas en el DataFrame que corresponden a las longitudes de onda redondeadas
  min_abs = df[df['Wavelength nm.'] == avg_min_wave].iloc[:,2:].reset_index(drop=True).T
  max_abs = df[df['Wavelength nm.'] == avg_max_wave].iloc[:,2:].reset_index(drop=True).T

  # Calculamos deltaAbs como la diferencia entre los absorbancias en las longitudes de onda redondeadas
  deltaAbs = max_abs - min_abs

  # Creamos un DataFrame de salida con los resultados
  out_df = deltaAbs
  out_df.insert(0, column ='',value=volume)
  out_df.reset_index(drop=True, inplace=True)
  out_df.columns= ['Volume µl', 'ΔAbs']
  # Añadimos dos columnas para poner los valores del minimo y máximo
  out_df["avg_min_wave (nm)"] = avg_min_wave
  out_df["avg_max_wave (nm)"] = avg_max_wave


  # Imprimimos los valores calculados
  print(f'Mínimo promedio:  ({avg_min_wave} nm)\t Máximo promedio:  ({avg_max_wave} nm)')


  return out_df



# Define the function to shift spectra so that they become zero at a specific wavelength

"""
## English
Adjusts all spectra so that their absorbance is zero at a given target wavelength.

Parameters:
wavelengths (dataframe): A dataframe of one column with the wavelength values.
absorbance (dataframe): A dataframe where each column is a spectrum at different volumes.
target_wavelength (float): The wavelength at which all spectra should be adjusted to zero.


shifted_absorbance: The absorbance values adjusted to zero at the target wavelength.

Returns:
shifted_data: A dataframe result of merging wavelengths and shifted_absorbance dataframes

## Español
Ajusta todos los espectro de manera que su absorbancua es cero a una determinada longitud
de onda.

Parámetros:
wavelengths (dataframe): Un dataframe de una columna con los valores de las longitudes de onda
absorbance (dataframe): Un dataframe en el que cada columna es un espectro a diferentes volúmenes.
target_wavelength (float): La longitudes de onda a la que todos los espectros se deben ajustar a cero.


shifted_absorbance: Los valores de absorbancia ajustados a cero a la longitud de onda elegida.

Resultado:
shifted_data: Un dataframe que se crea a partir de la unión de los dataframes 'wavelengths' y
shifted_absorbance.

"""
def shift_spectra_to_zero(datos, target_wavelength):
    wavelengths=datos.iloc[:,0]
    absorbance=datos.iloc[:,1:]

    # Find the index of the target wavelength in the wavelengths dataframe
    target_idx = (np.abs(wavelengths - target_wavelength)).argmin()

    # Extract the absorbance at the target wavelength for each spectrum
    absorbance_at_target = absorbance.iloc[target_idx, :]

    # Shift each spectrum by subtracting its absorbance at the target wavelength
    shifted_absorbance = absorbance - absorbance_at_target.values
    shifted_data = pd.concat([wavelengths, shifted_absorbance], axis=1)

    return shifted_data



########

# Definir la función de ajuste basada en el modelo matemático
def binding_model (params, v):
    # Combinamos ambos diccionarios (params)
    # params={**fixed_params, **variable_params}

    # Extraemos los parametros del dicionario combinado
    L = params['L']
    V0 = params['V0']
    Lo = params['Lo']
    Ro = params['Ro']
    Kd = params['Kd']
    epsilon = params['epsilon']

    Lt = Lo * v / (V0 + v)
    Rt = Ro * V0 / (V0 + v)
    deltaAbs = epsilon * L * (Lt + Rt + Kd - np.sqrt((Lt + Rt + Kd)**2 - 4 * Lt * Rt)) / 2
    return deltaAbs

In [ ]:
#@title Subir archivo y formato csv

# Subir el archivo y guardarlo en un diccionario
uploaded=files.upload()

# Obtener el nombre del arcivo subido al diccionario
file_name=list(uploaded.keys())[0]
# @markdown **Columnas intercaladas**

Intercaladas = False #@param {type:"boolean"}

# @markdown **Separador columnas**
Separador = "," #@param [",", ";"]
datos= leeFichero(nombrFich=file_name, intercaladas= Intercaladas, separador=Separador)
datos

In [ ]:
#@title PreprocesaAbsorbance
print("PreprocesaAbsorbance:")

DeltaAbs= preprocesaAbsorbance(datos)
DeltaAbs

In [ ]:
#@title Gráfico espectros
# @markdown En los espacios a continuación
# @markdown puedes escribir los títulos, ejes
# @markdown y titulo de leyenda del gráfico.


### Título del gráfico
Titulo = "Espectros a Diferentes Volúmenes de ***"  #@param {type:"string"}

### Título eje x
y_axis = "Absorbancia" #@param{type:"string"}

### Título eje y
x_axis = "Longitud de onda (nm)" #@param{type:"string"}

### Título leyenda
Leyenda = "Volumen (µL)" #@param{type:"string"}

### Opcíon para poner los espectros a 0 a una longitud de onda determinada
Shift_spectra= False #@param{type:"boolean"}
target_wavelength=600 #@param{type:"raw"}

shifted_absorbance = shift_spectra_to_zero(datos, target_wavelength)

if Shift_spectra == True:
  df = shifted_absorbance
else:
  df = datos
# Check the result
#shifted_absorbance[:]  # Displaying the first few rows of the shifted absorbance



# Output the plot directly in the notebook
output_notebook()

# Creamos una figura
p = figure(title=Titulo,
           x_axis_label=x_axis,
           y_axis_label=y_axis,
           width=1200, height=700)

# Definimos los tamaños de fuente para el titulo, los ejes y las etiquetas
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'




# Generamos una paleta de color personalizada usando Viridis256, con tantos colores como columna de datos haya
n_lines= len(df.columns[1:])
colors=linear_palette(Viridis256, n_lines)
#colors = Category20[max(3, min(20, n_lines))]  # Category20 supports up to 20 distinct colors


# Iteramos sobre las columnas (cada una representa un volumen differente)
for idx, col in enumerate(df.columns[1:]):
    p.line(df.iloc[:, 0], df[col], legend_label=f'{col}', line_width=2, color=colors[idx])

# Personalizamos la leyenda
p.legend.title = Leyenda
p.legend.location = "top_right"
p.legend.click_policy = "hide"  # Permite ocultar lineas haciendo click en la leyenda
p.toolbar_location = "below"
# Definimos los tamaños de fuente para la leyenda
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'


#Guardamos el gráfico como un objecto
spectra_plot_2D=p

# Mostramos el plot
show(spectra_plot_2D)




# Cargarmos nuestros datos experimentales
wavelengths = df.iloc[:, 0].values  # Wavelength (nm)

# Para evitar problemas asignamos 0 a "Baseline" y convertirmos el resto a números enteros
volumes = np.array([0 if col == 'Baseline' else int(col) for col in df.columns[1:]])  # Volumes (µL)

# Absorbancia para cada volumen (Baseline incluido)
absorbance = df.iloc[:, 1:].values

# Creamos una figura
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Representamos cada spectro de volumen como una línea 3D
for i, vol in enumerate(volumes):
    ax.plot(wavelengths, [vol] * len(wavelengths), absorbance[:, i], label=f'{vol} µL')

# Etiquetas y título
ax.set_xlabel(x_axis)
ax.set_ylabel(Leyenda)
ax.set_zlabel(y_axis)
ax.set_title(Titulo)

#Guardamos el gráfico como un objecto
spectra_plot_3D=fig

# Mostramos el gráfico
spectra_plot_3D.show()
#plt.show()

In [ ]:
#@title Gráfico Δ Absorbancia vs Volumen (µL)


### Título del gráfico
Titulo = "Δ Absorbancia en función del volumen añadido ***"  #@param {type:"string"}

### Título eje x
y_axis = "Δ Absorbancia" #@param{type:"string"}

### Título eje y
x_axis = "Volumen (µL)" #@param{type:"string"}

### Título leyenda
Leyenda = "Δ Absorbancia" #@param{type:"string"}


# Display Bokeh plots in the notebook
output_notebook()

# Creamos una figura
p = figure(width=1200, height=700, title=Titulo)

# Añadimos un diagrama de dispersión de puntos
scatter = p.scatter(x=DeltaAbs.iloc[:, 0],y=DeltaAbs.iloc[:, 1] ,  legend_label=Leyenda, color='blue', size=10)

# Personalizamos los títulos de los ejes
p.xaxis.axis_label = x_axis
p.yaxis.axis_label = y_axis



# Definimos los tamaños de fuente para el titulo, los ejes y las etiquetas
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

# Añadimos la leyenda
p.legend.title = 'Legend'
# Definimos los tamaños de fuente para la leyenda
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'
p.legend.location = "bottom_right"
p.toolbar_location = "below"


#Guardamos el gráfico como un objecto
D_Absorbance_plot=p # Este no seríe necesario guardarlo

# Mostramos el plot
show(D_Absorbance_plot)



In [ ]:
# @title Parámetros
# @markdown **Parámetros fijos y a optimizar**
# @markdown Selecciona los parametros fijos y aquellos a optimizar:

import numpy as np
# @markdown

# @markdown **- L [cm] (longitud trayectoria luz)**
# Usamos un desplegable para elergir si los parametros deben ser fijados o optimizados
L_fixed = "Yes"  # @param ["Yes", "No"]
L = 1  # @param {type:"number"}

# @markdown

# @markdown **- V0 [µL] (volumen solución inicial)**
V0_fixed = "Yes"  # @param ["Yes", "No"]
V0 = 1000  # @param {type:"number"}

# @markdown

# @markdown **- L0 [µM] (concentración solución inicial ApoMgb - Ligando)**
Lo_fixed = "Yes"  # @param ["Yes", "No"]
Lo = 55  # @param {type:"number"}

# @markdown

# @markdown **- R0 [µM] (concentración solución inicial Hemoglobina - Receptor)**
Ro_fixed = "No"  # @param ["Yes", "No"]
Ro = 2  # @param {type:"number"}

# @markdown

# @markdown **- Kd**
Kd_fixed = "No"  # @param ["Yes", "No"]
Kd = 0.1  # @param {type:"number"}

# @markdown

# @markdown **- epsilon**
epsilon_fixed = "No"  # @param ["Yes", "No"]
epsilon = 0.1  # @param {type:"number"}

# Inicializamos listas para los parametros fijos y variables (estos últimos a optimizar después)
fixed_params = {}
variable_params = {}

if L_fixed == "Yes":
    fixed_params['L'] = L
else:
    variable_params['L'] = L

if V0_fixed == "Yes":
    fixed_params['V0'] = V0
else:
    variable_params['V0'] = V0

if Lo_fixed == "Yes":
    fixed_params['Lo'] = Lo
else:
    variable_params['Lo'] = Lo

if Ro_fixed == "Yes":
    fixed_params['Ro'] = Ro
else:
    variable_params['Ro'] = Ro

if Kd_fixed == "Yes":
    fixed_params['Kd'] = Kd
else:
    variable_params['Kd'] = Kd

if epsilon_fixed == "Yes":
    fixed_params['epsilon'] = epsilon
else:
    variable_params['epsilon'] = epsilon

# Output de los parametros fijos y variables
print("Fixed Parameters:")
for key, value in fixed_params.items():
    print(f"{key} = {value}")

print("\nVariable Parameters:")
for key, value in variable_params.items():
    print(f"{key} = {value}")


In [ ]:
#@title Procesa
# Preparamos la lista de los nombres de parametros a optimizar
nombrParVar = list(variable_params.keys())

# Valores independientes (v) y dependientes (Absorbancia)
fKwargs = dict(v = DeltaAbs.iloc[:, 0].values) # Datos de volumen ApoMgb añadido
deltaAbs_exp = DeltaAbs.iloc[:, 1].values #Datos de absorbancia

initial_params= {**fixed_params, **variable_params}
cotaInf = [0 for param in nombrParVar]
cotaSup= [2 if param =='Ro' else np.inf for param in nombrParVar]

sol=procesa(argLeastSquares = argLeastSquares,
                     dictParEstim = initial_params,
                     nombrParVar = nombrParVar,
                     f = binding_model,
                     fKwargs = fKwargs,
                     Y = deltaAbs_exp,
                     bounds =[cotaInf, cotaSup])


In [ ]:
# @title Gráfico Δ Absorbancia vs Volumen (µL) con el ajuste del modelo



### Título del gráfico
Titulo = "Δ Absorbancia en función del volumen añadido ***"  #@param {type:"string"}

### Título eje x
y_axis = "Δ Absorbancia" #@param{type:"string"}

### Título eje y
x_axis = "Volumen (µL)" #@param{type:"string"}

### Título leyenda
Leyenda = "Δ Absorbancia" #@param{type:"string"}


# Display Bokeh plots in the notebook
output_notebook()

# Simulación de datos usando los parámetros optimizados
v = DeltaAbs.iloc[:, 0].values
fitted_deltaAbs = binding_model(sol['parAjustados'], v) # Datos de volumen
deltaAbs_exp= DeltaAbs.iloc[:,1].values

# Creamos un dataframe para guardar los valores de fitted_DeltaAbs
DeltaAbs_model=DeltaAbs.copy() # Hacemos una copia de DeltaAbs
DeltaAbs_model['ΔAbs']= fitted_deltaAbs # Asignamos los valores de ΔAbs del modelo




# Creamos una figura
p = figure(width=1200, height=700, title=Titulo)

# Añadimos un gráfico de dispersión de puntos (datos experimentales)
scatter = p.scatter(x=v,y=deltaAbs_exp ,  legend_label=Leyenda, color='blue', size=10)

# Añadimos un gráfico de líneas
p.line(x=v, y=fitted_deltaAbs, legend_label='Ajuste del modelo', color='red', line_width=3)

# Definimos los títulos de los ejes
p.xaxis.axis_label = x_axis
p.yaxis.axis_label = y_axis

# Definimos los tamaños de fuente para el titulo, los ejes y las etiquetas
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

# Añadimos la leyenda
p.legend.title = 'Legend'
# Definimos los tamaños de fuente para la leyenda
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'
p.legend.location = "bottom_right"
p.toolbar_location = "below"

#Guardamos el gráfico como un objecto
D_Absorbance_plot_fitted=p # Este no seríe necesario guardarlo

# Mostramos el plot
show(D_Absorbance_plot_fitted)


In [ ]:
#@title Guardado y generación de zip de descarga de datos
# Ahora vamos a recoger todos los datos que hemos introducido y producido y exportarlos

# Datos iniciales
datos.to_csv('Experimental_data.csv', index=False)

# Datos de Δ Absorbancia vs volumen experimentales
DeltaAbs.to_csv('DeltaAbs_expertimental.csv', index=False)

# Datos de Δ Absorbancia vs volumen del ajuste del modelo
DeltaAbs_model.to_csv('DeltaAbs_fitted.csv', index=False)



# Ahora vamos a guardar los gráficos

# Gráfico de los espectros 2D
output_file("spectra_plot_2D.html")
save(spectra_plot_2D, title="Bokeh plot",)

# Gráfico de los espectros 3D
spectra_plot_3D.savefig('spectra_plot_3D.png', format='png', dpi=400) # Ajustar dpi para ajustar la resolución

# Gráfico Δ Absorbancia vs volumen experimentales con el ajuste del modelo
output_file("D_Absorbance_plot_fitted.html")
save(D_Absorbance_plot_fitted, title="Bokeh plot",)


# A continuación editamos los datos generados por el ajuste para guardarlos y exportarlos

# Para guardar los resultados vamos a crear dos diccionarios a partir de la solución de procesa
# y de los diccionarios de los parametros iniciales (fijos y variables)

# Lista de claves a extraer del dictionario sol
keys= ['parAjustados', 'sdPar', 'R2', 'detalles']

partial= {key:sol[key] for key in keys}

Initial_params={'initialPar':initial_params}

results={**Initial_params, **partial}

print(results)

# Create CSV
with open('Fitting_result.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Headers
    writer.writerow(['initialPar', '', '', 'parAjustados', '', '', 'sdPar'])

    # Row 1: L, V0, Lo
    writer.writerow([
        'L', results['initialPar']['L'], '', 'Ro', results['parAjustados']['Ro'], '', 'Ro_std', results['sdPar']['Ro_std']
    ])
    writer.writerow([
        'V0', results['initialPar']['V0'], '', 'Kd', results['parAjustados']['Kd'], '', 'Kd_std', results['sdPar']['Kd_std']
    ])
    writer.writerow([
        'Lo', results['initialPar']['Lo'], '', 'epsilon', results['parAjustados']['epsilon'], '', 'epsilon_std', results['sdPar']['epsilon_std']
    ])
    writer.writerow([
        'Ro', results['initialPar']['Ro'], '', 'L', results['parAjustados']['L'], '', ''
    ])
    writer.writerow([
        'Kd', results['initialPar']['Kd'], '', 'V0', results['parAjustados']['V0'], '', ''
    ])
    writer.writerow([
        'epsilon', results['initialPar']['epsilon'], '', 'Lo', results['parAjustados']['Lo'], '', ''
    ])

    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # R2 Section
    writer.writerow(['R2'])
    writer.writerow(['R2', results['R2']])
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Detalles Section
    writer.writerow(['Detalles'])

    # x values (convert floats to strings to concatenate with 'x')
    writer.writerow(['x'] + [str(x) for x in results['detalles']['x']])

    # cost
    writer.writerow(['cost', results['detalles']['cost']])

    # Add blank rows
    writer.writerow([''] * 7)

    # fun values (split into multiple rows)
    fun_values = results['detalles']['fun']
    writer.writerow(['fun'] + [str(f) for f in fun_values[:6]])
    writer.writerow(['fun'] + [str(f) for f in fun_values[6:]])

    # Add blank rows
    writer.writerow([''] * 7)

    # jac values (split into multiple rows)
    for row in results['detalles']['jac']:
        writer.writerow(['jac'] + [str(v) for v in row])

    # Add blank rows
    writer.writerow([''] * 7)

    # grad values (convert floats to strings)
    writer.writerow(['grad'] + [str(g) for g in results['detalles']['grad']])

    # optimality
    writer.writerow(['optimality', results['detalles']['optimality']])

    # active_mask (convert integers to strings)
    writer.writerow(['active_mask'] + [str(a) for a in results['detalles']['active_mask']])

    # nfev, njev, status, message, success
    writer.writerow(['nfev', results['detalles']['nfev']])
    writer.writerow(['njev', results['detalles']['njev']])
    writer.writerow(['status', results['detalles']['status']])
    writer.writerow(['message', results['detalles']['message']])
    writer.writerow(['success', results['detalles']['success']])


# Ahora procedemos a guardar todos los archivos y comprimirlos en un zip

# Tomamos nota de la fecha y hora actuales
current_time = datetime.now().strftime("%d%m%Y%H%M%S")

#Definimos el prefijo y creamos el nombre completo del archivo zip
name ="spectra_" #@param {type: "string"}
zip_filename = f"{name}{current_time}.zip"

# Creamos el archivo Zip con el nombre elegido
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Add CSV files
    zipf.write('Experimental_data.csv')
    zipf.write('DeltaAbs_expertimental.csv')
    zipf.write('DeltaAbs_fitted.csv')
    zipf.write ('Fitting_result.csv')
    # Add HTML files (Bokeh plots)
    zipf.write('spectra_plot_2D.html')
    zipf.write('D_Absorbance_plot_fitted.html')

    # Add PNG file (3D spectra plot)
    zipf.write('spectra_plot_3D.png')




# Download the zipped file
files.download(zip_filename)
